# Myelin Mapping Notebook

These workflows process T1 and T2 weighted images to then be used to quantify cortical myelin content per methods described in Gordon et al., 2017 (doi: 10.1016/j.neuron.2017.07.011)

In [1]:
# Import all othe functions/nodes being used
from nipype.pipeline.engine import Workflow, Node, MapNode
from nipype.interfaces.utility import IdentityInterface, Function
from nipype.interfaces.io import SelectFiles, DataSink, DataGrabber, FreeSurferSource
from nipype.interfaces.fsl import FSLCommand, BET, Reorient2Std, FAST
from nipype.interfaces.freesurfer import BBRegister, Binarize, ReconAll
from nipype.interfaces.ants import N4BiasFieldCorrection
from pandas import read_csv

# Declare study-specific variables
setup='Jay'

if setup=='Jay':
    study_home = '/Users/dendrite/Desktop/Jay'
    raw_data = study_home + '/raw'
    subject_ids = '3000'
elif setup=='Cat':
    study_home = '/moochie/user_data/CamachoCat/5YOP'
    raw_data = '/moochie/study_data/5YOP/MRI_processing'
    temp_fs_dir = '/home/camachocm2/Analysis/5YOP/freesurfer'
    session_info = read_csv(study_home + '/misc/session_info.csv',index_col=None)
    subject_ids = session_info['subject_id'].unique().tolist()
    subject_ids = map(str,subject_ids)
    
output_dir = study_home + '/proc'
workflow_dir = study_home + '/workflows'
fs_dir = study_home + '/freesurfer'


# Set any processing settings
FSLCommand.set_default_output_type('NIFTI_GZ')

In [2]:
# Identity node- select subjects
infosource = Node(IdentityInterface(fields=['subject_id']),
                     name='infosource')
infosource.iterables = ('subject_id', subject_ids)

# Selectfiles node to grab subject T2w data
if setup=='Jay':
    t2w_templates = {'T2anat': raw_data + '/t2w_2.nii.gz'}
elif setup=='Cat':
    t2w_templates = {'T2anat': raw_data + '/{subject_id}/ANAT_NIFTIS/t2w_2.nii.gz'}
t2w_source = Node(SelectFiles(t2w_templates), name='t2w_source')

# datasink node
datasink = Node(DataSink(base_directory=output_dir), name='datasink')

## Preprocess T1w data

In [3]:
# grab raw T1-weighted images and process through freesurfer
t1w_template = {'t1_anats':raw_data + '/%s/ANAT_NIFTIS/t1w_*.nii.gz'}
t1w_source = Node(DataGrabber(base_directory = raw_data, 
                              field_template = t1w_template,
                              template=raw_data + '/%s/ANAT_NIFTIS/t1w_*.nii.gz',
                              sort_filelist=True,
                              in_fields=['subject_id'],
                              outfields=['t1_anats'],
                              template_args={'t1_anats':[['subject_id']]}), 
                  name='t1w_source')

#process through recon-all
reconall = Node(ReconAll(directive='all',flags='-gcut',openmp=4,
                         subjects_dir=fs_dir,use_T2=True), name='reconall')

In [ ]:
fs_proc = Workflow(name='freesurfer_proc_flow')
fs_proc.connect([(infosource, t1w_source, [('subject_id','subject_id')]),
                 (infosource, t2w_source, [('subject_id','subject_id')]),
                 (infosource, reconall, [('subject_id','subject_id')]),
                 (t1w_source, reconall, [('t1_anats','T1_files')]), 
                 (t2w_source, reconall, [('T2anat','T2_file')])
                ])
fs_proc.base_dir = workflow_dir
fs_proc.write_graph(graph2use='flat')
fs_proc.run('MultiProc', plugin_args={'n_procs': 10, 'memory_gb': 20})

## Preprocess T2w data
The nodes and workflow below preprocess T2-weighted images to match T1-weighted data processed externally using FreeSurfer v6.

In [7]:
#Function to quantify T1/T2 ratio
def quantify_ratio(T2_nifti):
    from nibabel import load, save, Nifti1Image
    from numpy import divide, zeros
    
    #import images
    t1w_img = load('/Users/dendrite/Desktop/Jay/freesurfer/3000/mri/T1.mgz')
    t2w_img = load(T2_nifti)
    t1w_data = t1w_img.get_data()
    t2w_data = t2w_img.get_data()
    
    #calculate ratio and create new image
    ratio_data = divide(t1w_data, t2w_data)
    ratio_img = Nifti1Image(ratio_data, header=t2w_img.header, affine=t2w_img.affine)
    save(ratio_img, 'ratio_image.nii.gz')
    return ratio_img

#test
#quantify_ratio(study_home + '/workflows/preprocess_t2/register/registered_t2w.nii.gz')

In [8]:
# FreeSurferSource node to grab processed T1w data from freesurfer
t1w_source = Node(FreeSurferSource(subjects_dir=fs_dir), name='t1w_source')

# skullstrip the T2w using BET (FSL)
t2strip = Node(BET(), name='t2strip')

# N4 bias correction of the T2w data (ANTs)
bias_correct = Node(N4BiasFieldCorrection(),name='bias_correct')

# Reorient T2w to standard
reorient_t2w = Node(Reorient2Std(), name='reorient_t2w')

# Register T2w to T1w using FreeSurfer's Boundary-Based Registration
register = Node(BBRegister(subjects_dir = fs_dir, contrast_type='t2'), name='register')

#Calculate T1/T2 ratio
quantify_ratio = Node(Function(input_names=['T2_nifti'], 
                           output_names=['ratio_img'], 
                           function=quantify_ratio), name='quantify_ratio')


In [9]:
# Write the workflow
preprocess_t2 = Workflow(name='preprocess_t2')
preprocess_t2.connect([(infosource, t1w_source, [('subject_id','subject_id')]),
                       (infosource, t2w_source, [('subject_id','subject_id')]),
                       (infosource, register, [('subject_id','subject_id')]),
                       (t2w_source, t2strip,[('T2anat','in_file')]),
                       (t2strip, reorient_t2w,[('out_file','in_file')]),
                       (reorient_t2w, bias_correct,[('out_file','input_image')]),
                       (bias_correct, register,[('output_image','source_file')]),
                       (bias_correct, quantify_ratio,[('output_image','T2_nifti')]),
                       (t2strip, datasink,[('out_file','skullstripped_t2w')]),
                       (register, datasink,[('registered_file','registered_t2w')])
                       ])

preprocess_t2.base_dir = workflow_dir
preprocess_t2.write_graph(graph2use='flat')
preprocess_t2.run('MultiProc', plugin_args={'n_procs': 2, 'memory_gb': 10})

200118-22:20:16,974 workflow INFO:
	 Generated workflow graph: /Users/dendrite/Desktop/Jay/workflows/preprocess_t2/graph.png (graph2use=flat, simple_form=True).
200118-22:20:17,2 workflow INFO:
	 Workflow preprocess_t2 settings: ['check', 'execution', 'logging', 'monitoring']
200118-22:20:17,35 workflow INFO:
	 Running in parallel.
200118-22:20:17,40 workflow INFO:
	 [MultiProc] Running 0 tasks, and 8 jobs ready. Free memory (GB): 10.00/10.00, Free processors: 2/2.
200118-22:20:17,104 workflow INFO:
	 [Node] Setting-up "preprocess_t2.t2w_source" in "/Users/dendrite/Desktop/Jay/workflows/preprocess_t2/_subject_id_0/t2w_source".200118-22:20:17,105 workflow INFO:
	 [Node] Setting-up "preprocess_t2.t1w_source" in "/Users/dendrite/Desktop/Jay/workflows/preprocess_t2/_subject_id_0/t1w_source".

200118-22:20:17,121 workflow INFO:
	 [Node] Running "t1w_source" ("nipype.interfaces.io.FreeSurferSource")200118-22:20:17,121 workflow INFO:
	 [Node] Running "t2w_source" ("nipype.interfaces.io.Select

200118-22:20:29,57 workflow INFO:
	 [MultiProc] Running 0 tasks, and 6 jobs ready. Free memory (GB): 10.00/10.00, Free processors: 2/2.
200118-22:20:29,94 workflow INFO:
	 [Node] Setting-up "preprocess_t2.register" in "/Users/dendrite/Desktop/Jay/workflows/preprocess_t2/_subject_id_0/register".200118-22:20:29,92 workflow INFO:
	 [Node] Setting-up "preprocess_t2.quantify_ratio" in "/Users/dendrite/Desktop/Jay/workflows/preprocess_t2/_subject_id_0/quantify_ratio".

200118-22:20:29,100 workflow INFO:
	 [Node] Running "quantify_ratio" ("nipype.interfaces.utility.wrappers.Function")200118-22:20:29,101 workflow INFO:
	 [Node] Running "register" ("nipype.interfaces.freesurfer.preprocess.BBRegister"), a CommandLine Interface with command:
bbregister --t2 --reg /Users/dendrite/Desktop/Jay/workflows/preprocess_t2/_subject_id_0/register/t2w_2_brain_reoriented_corrected_bbreg_0.dat --mov /Users/dendrite/Desktop/Jay/workflows/preprocess_t2/_subject_id_0/register/t2w_2_brain_reoriented_corrected.nii

bbregister --t2 --reg /Users/dendrite/Desktop/Jay/workflows/preprocess_t2/_subject_id_0/register/t2w_2_brain_reoriented_corrected_bbreg_0.dat --mov /Users/dendrite/Desktop/Jay/workflows/preprocess_t2/_subject_id_0/register/t2w_2_brain_reoriented_corrected.nii.gz --s 0
200118-22:20:39,283 workflow WARNING:
	 [Node] Error on "preprocess_t2.register" (/Users/dendrite/Desktop/Jay/workflows/preprocess_t2/_subject_id_0/register)
200118-22:20:39,442 workflow WARNING:
	 [Node] Error on "preprocess_t2.quantify_ratio" (/Users/dendrite/Desktop/Jay/workflows/preprocess_t2/_subject_id_0/quantify_ratio)
200118-22:20:41,78 workflow ERROR:
	 Node quantify_ratio.a1 failed to run on host dendrites-iMac.local.
200118-22:20:41,79 workflow ERROR:
	 Saving crash info to /Users/dendrite/Desktop/Jay/5yop/crash-20200118-222041-dendrite-quantify_ratio.a1-78f45037-b313-4646-a653-2060cd910b2e.pklz
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packa

200118-22:20:45,94 workflow INFO:
	 [MultiProc] Running 0 tasks, and 0 jobs ready. Free memory (GB): 10.00/10.00, Free processors: 2/2.
200118-22:20:47,81 workflow INFO:
	 ***********************************
200118-22:20:47,82 workflow ERROR:
	 could not run node: preprocess_t2.quantify_ratio.a3
200118-22:20:47,83 workflow INFO:
	 crashfile: /Users/dendrite/Desktop/Jay/5yop/crash-20200118-222027-dendrite-quantify_ratio.a3-081c8982-86ba-4609-ab7d-331a20d31a81.pklz
200118-22:20:47,83 workflow ERROR:
	 could not run node: preprocess_t2.register.a3
200118-22:20:47,84 workflow INFO:
	 crashfile: /Users/dendrite/Desktop/Jay/5yop/crash-20200118-222027-dendrite-register.a3-8e996924-3061-4df6-9a43-8fd8f5850180.pklz
200118-22:20:47,85 workflow ERROR:
	 could not run node: preprocess_t2.quantify_ratio.a2
200118-22:20:47,85 workflow INFO:
	 crashfile: /Users/dendrite/Desktop/Jay/5yop/crash-20200118-222031-dendrite-quantify_ratio.a2-f43015a7-c4bd-48e2-ae5f-9fc7a0c13aab.pklz
200118-22:20:47,86 workf

RuntimeError: Workflow did not execute cleanly. Check log for details

## Resample surfaces

In [ ]:
def resample_surfs(surfs):
    return(resampled_surfs)

def volume_to_surface_myelin(volume):
    return(surface_map)


## Perform T2 surface registration and compute T1/T2 ratio